In [ ]:
import sys
import math
import random
from common import print_tour, read_input
    

# 2点間の距離を計算
def distance(city1, city2):
    return math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)



# greedy
def solve_greedy(cities):

    N = len(cities)

    dist = [[0] * N for i in range(N)]
    for i in range(N):
        for j in range(i, N):
            dist[i][j] = dist[j][i] = distance(cities[i], cities[j])

    shortest = 100000
    
    # スタート地点を変えて一番短いものを返す
    # Challenge6は全部やると大きすぎたのでランダムにk個選ぶ
    if N<=512:
        size = range(N)
    else:
        size = random.sample(range(N), k=30)
        print(size)


    for i in size:
        current_city = i
        unvisited_cities = set(range(0, N))
        unvisited_cities.remove(current_city)
        tour = [current_city]

        while unvisited_cities:
            next_city = min(unvisited_cities,
                            key=lambda city: dist[current_city][city])
            unvisited_cities.remove(next_city)
            tour.append(next_city)
            current_city = next_city
        #距離計算
        path_length = sum(dist[tour[i]][tour[(i + 1) % N]] for i in range(N))
        
        # 最短距離更新
        if path_length < shortest:
            shortest = path_length
            shortest_tour = tour

    # 配列を0始まりにする
    for i in range(N):
        if shortest_tour[0] == 0:break
        if shortest_tour[i] == 0:
            shortest_tour = shortest_tour[i:N]+shortest_tour[:i]
            break
            

    return shortest_tour,dist


        

# intersectionで使う
def t(a,b,c):
    return (a[0]-b[0])*(c[1]-a[1])+(a[1]-b[1])*(a[0]-c[0])

# 「city1とcity2を結ぶ線分」と「city3とcity4を結ぶ線分」が交差しているか
def intersection(city1,city2,city3,city4):
    t1 = t(city1,city2,city3)
    t2 = t(city1,city2,city4)
    t3 = t(city3,city4,city1)
    t4 = t(city3,city4,city2)
    return (t1*t2<0) and (t3*t4<0)

# 2-opt
def opt2(tour,cities):

    N = len(tour)
    flag = True

    # 交差が完全になくなるまで繰り返す
    while flag:

        # 交差が見つからなければflagはFalseのまま→ループ終了
        flag = False
        
        # 交差してる線分があるか全探索
        for i in range(N-2):
            for j in range(i+2, N):
                # 交差が見つかったら入れ替え
                if intersection(cities[tour[i]],cities[tour[i+1]],cities[tour[j]],cities[tour[(j+1)%N]]):
                    flag = True
                    # i+1とjを入れ替え
                    tmp = tour[i+1]
                    tour[i+1] = tour[j]
                    tour[j%N] = tmp
                    # i+2 → jまでの順路が逆になる
                    tour[i+2:j] = reversed(tour[i+2:j])
                    break
             
    return tour

# 線分の中に一つの点を挿入してみてより短くなれば更新
def or_opt(tour,dist):
    N = len(tour)
    flag = True
    while flag:
        flag = False
        for i in range(1,N):
            for j in range(N):
                if i==j or i==(j+1)%N:continue
                pre_dist = dist[tour[i]][tour[(i-1)%N]]+dist[tour[i]][tour[(i+1)%N]]+dist[tour[j]][tour[(j+1)%N]]
                new_dist = dist[tour[i]][tour[j]]+dist[tour[i]][tour[(j+1)%N]]+dist[tour[(i-1)%N]][tour[(i+1)%N]]
                if pre_dist > new_dist:
                    tour.insert(j+1,tour[i])
                    if i<j:
                        del tour[i]
                    else:
                        del tour[i+1]
                    break
    return tour
                
    

if __name__ == '__main__':
    assert len(sys.argv) > 1
    cities = read_input(sys.argv[1])
    
    # greedy
    (tour,dist) = solve_greedy(cities)
    # 2_opt
    tour = opt2(tour,cities)
    # or_opt
    tour = or_opt(tour,dist)
    
    print_tour(tour)


In [ ]:
import sys
import math

from common import print_tour, read_input


def distance(city1, city2):
    return math.sqrt((city1[0] - city2[0]) ** 2 + (city1[1] - city2[1]) ** 2)


def random_greedy_solve(cities):
    
    N = len(cities)
    path_length = 0
    shortest_path = 1000000
    shortest_tour = []

    dist = [[0] * N for i in range(N)]
    for i in range(N):
        for j in range(i, N):
            dist[i][j] = dist[j][i] = distance(cities[i], cities[j])

    for i in range(N):#pick differnet start point
        current_city = i
        unvisited_cities = set(range(0, N))
        unvisited_cities.remove(i)
        tour = [current_city]

        while unvisited_cities:
            next_city = min(unvisited_cities,key=lambda city: dist[current_city][city])
            path_length = dist[current_city][next_city]
            unvisited_cities.remove(next_city)
            tour.append(next_city)
            current_city = next_city
            
        if path_length < shortest_path:
            shortest_path = path_length
            shortest_tour = tour
        
        return shortest_tour

"""
I only checked whether the neighbourhood/following routes are intersected or not.
e.g tour[0:4] 
But I think there is also possibility that [1]->[2] and [7]->[8] are intersected. 
However, if seraching the whole pairs of route, it would take too much time.
So I want to  ask if there is any better solution rather than randomly checking. 
"""
def opt_2(tour,cities):
    
    

